In [38]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [39]:
from pyspark.sql import SparkSession


spark = SparkSession.builder\
    .master("local[2]")\
    .appName("Lesson_5")\
    .config("spark.executor.instances",2)\
    .config("spark.executor.memory",'2g')\
    .config("spark.executor.cores",1)\
    .getOrCreate()
    
sc = spark.sparkContext

# Оконные функции

Оконная функция - функция, которая работает с выделенным набором строк (окном, партицией) и выполняет вычисление для этого набора строк в отдельном столбце. 

Партиции (окна из набора строк) - это набор строк, указанный для оконной функции по одному из столбцов или группе столбцов таблицы. Партиции для каждой оконной функции в запросе могут быть разделены по различным колонкам таблицы.

In [40]:
# Предположим, что есть таблицы:
from pyspark.sql import functions as F
from pyspark.sql.types import *

data =[
    ("Banana",1000,"USA"), ("Carrots",1500,"USA"), ("Beans",1600,"USA"),\
    ("Orange",2000,"USA"), ("Orange",2000,"USA"), ("Banana",400,"China"),\
    ("Carrots",1200,"China"), ("Beans",1500,"China"), ("Orange",4000,"China"),\
    ("Banana",2000,"Canada"), ("Carrots",2000,"Canada"), ("Beans",2000,"Mexico")
    ]
columns = ["Product","Amount", "Country"]

df = spark.createDataFrame(data=data, schema=columns)
df.printSchema()
df.show(truncate=False) # truncate - обрезать ли длинные строки

root
 |-- Product: string (nullable = true)
 |-- Amount: long (nullable = true)
 |-- Country: string (nullable = true)

+-------+------+-------+
|Product|Amount|Country|
+-------+------+-------+
|Banana |1000  |USA    |
|Carrots|1500  |USA    |
|Beans  |1600  |USA    |
|Orange |2000  |USA    |
|Orange |2000  |USA    |
|Banana |400   |China  |
|Carrots|1200  |China  |
|Beans  |1500  |China  |
|Orange |4000  |China  |
|Banana |2000  |Canada |
|Carrots|2000  |Canada |
|Beans  |2000  |Mexico |
+-------+------+-------+



In [41]:
# в sql оконная функция записывается следующим образом
df.registerTempTable('df')

spark.sql('''
select *, 
row_number() over( partition by Country order by Amount ) as rn from df
''').show()

+-------+------+-------+---+
|Product|Amount|Country| rn|
+-------+------+-------+---+
| Banana|  2000| Canada|  1|
|Carrots|  2000| Canada|  2|
| Banana|   400|  China|  1|
|Carrots|  1200|  China|  2|
|  Beans|  1500|  China|  3|
| Orange|  4000|  China|  4|
|  Beans|  2000| Mexico|  1|
| Banana|  1000|    USA|  1|
|Carrots|  1500|    USA|  2|
|  Beans|  1600|    USA|  3|
| Orange|  2000|    USA|  4|
| Orange|  2000|    USA|  5|
+-------+------+-------+---+



In [42]:
# в спарке следующим образом
from pyspark.sql import Window


windSpec = Window()\
    .partitionBy('Country')\
    .orderBy('Amount')
    # .rowsBetween(Window.unboundedPreceding, Window.currentRow - 1)

df.withColumn('rn', F.row_number().over(windSpec)).show()

+-------+------+-------+---+
|Product|Amount|Country| rn|
+-------+------+-------+---+
| Banana|  2000| Canada|  1|
|Carrots|  2000| Canada|  2|
| Banana|   400|  China|  1|
|Carrots|  1200|  China|  2|
|  Beans|  1500|  China|  3|
| Orange|  4000|  China|  4|
|  Beans|  2000| Mexico|  1|
| Banana|  1000|    USA|  1|
|Carrots|  1500|    USA|  2|
|  Beans|  1600|    USA|  3|
| Orange|  2000|    USA|  4|
| Orange|  2000|    USA|  5|
+-------+------+-------+---+



# Самостоятельная работа к уроку
На уроке мы попробовали оконные и пользовательские функции. Теперь закрепим полученные знания.

## Данные: [google drive: raw_sales.csv](https://drive.google.com/file/d/1G2N7Mnt4-Tqz4JdJxutGDMbJiOr32kZp/view?usp=sharing)

 Каждая строчка это продажа жилья, которая состоит из следующих полей (думаю описание не требуется):
*   date of sale
*   price
*   property type
*   number of bedrooms
*   4digit postcode

In [43]:
!wget 'https://drive.google.com/uc?export=download&id=1xbtFBPz50OBoyYFVGd-B03pCTJdCax07' -O raw_sales.csv

--2022-10-03 11:53:10--  https://drive.google.com/uc?export=download&id=1xbtFBPz50OBoyYFVGd-B03pCTJdCax07
Resolving drive.google.com (drive.google.com)... 142.251.45.110, 2607:f8b0:4004:83f::200e
Connecting to drive.google.com (drive.google.com)|142.251.45.110|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0c-5k-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/ttqsi7hbco695j2i3ftlm4d08hmdfp6o/1664797950000/04099736791713398091/*/1xbtFBPz50OBoyYFVGd-B03pCTJdCax07?e=download&uuid=3f501c4e-4936-467d-a01f-9ed0e2c2d63f [following]
--2022-10-03 11:53:11--  https://doc-0c-5k-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/ttqsi7hbco695j2i3ftlm4d08hmdfp6o/1664797950000/04099736791713398091/*/1xbtFBPz50OBoyYFVGd-B03pCTJdCax07?e=download&uuid=3f501c4e-4936-467d-a01f-9ed0e2c2d63f
Resolving doc-0c-5k-docs.googleusercontent.com (doc-0c-5k-docs.googleusercontent.com)... 172.253.115.132, 2607:f8b0:4004:

In [169]:
df = spark.read.csv('raw_sales.csv', header=True, inferSchema=True)
df.printSchema()
df.show(truncate=False)

root
 |-- datesold: timestamp (nullable = true)
 |-- postcode: integer (nullable = true)
 |-- price: integer (nullable = true)
 |-- propertyType: string (nullable = true)
 |-- bedrooms: integer (nullable = true)

+-------------------+--------+-------+------------+--------+
|datesold           |postcode|price  |propertyType|bedrooms|
+-------------------+--------+-------+------------+--------+
|2007-02-07 00:00:00|2607    |525000 |house       |4       |
|2007-02-27 00:00:00|2906    |290000 |house       |3       |
|2007-03-07 00:00:00|2905    |328000 |house       |3       |
|2007-03-09 00:00:00|2905    |380000 |house       |4       |
|2007-03-21 00:00:00|2906    |310000 |house       |3       |
|2007-04-04 00:00:00|2905    |465000 |house       |4       |
|2007-04-24 00:00:00|2607    |399000 |house       |3       |
|2007-04-30 00:00:00|2606    |1530000|house       |4       |
|2007-05-24 00:00:00|2902    |359000 |house       |3       |
|2007-05-25 00:00:00|2906    |320000 |house       |3   

## Задание 1
Добавьте к таблице следующие поля:
*  Средняя стомость 10 проданных домов до текущего в том же районе (4digit postcode)
*  Средняя стомость 10 проданных домов после текущего в том же районе (4digit postcode)
*  Стоимость последнего проданного дома до текущего


In [71]:
from pyspark.sql import Window
import pyspark.sql.functions as F

# Средняя стомость 10 проданных домов до текущего в том же районе (в SQL запросе)
df.registerTempTable('df_task_1_1')

spark.sql('''
select *, 
AVG(price) OVER(PARTITION BY postcode ORDER BY datesold ROWS BETWEEN 9 PRECEDING AND CURRENT ROW) AS avg_10_prev
FROM df_task_1_1
''').show(15)

/usr/local/lib/python3.7/dist-packages/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


+-------------------+--------+-------+------------+--------+-----------------+
|           datesold|postcode|  price|propertyType|bedrooms|      avg_10_prev|
+-------------------+--------+-------+------------+--------+-----------------+
|2007-07-08 00:00:00|    2600| 327000|       house|       1|         327000.0|
|2007-08-16 00:00:00|    2600| 790000|       house|       4|         558500.0|
|2007-12-05 00:00:00|    2600| 825000|       house|       3|647333.3333333334|
|2008-01-21 00:00:00|    2600| 315000|        unit|       1|         564250.0|
|2008-04-24 00:00:00|    2600| 292500|       house|       1|         509900.0|
|2008-05-30 00:00:00|    2600| 329000|        unit|       2|         479750.0|
|2008-06-19 00:00:00|    2600| 765000|       house|       5|         520500.0|
|2008-07-29 00:00:00|    2600| 927000|       house|       4|         571312.5|
|2008-09-02 00:00:00|    2600|1380000|       house|       5|661166.6666666666|
|2008-09-08 00:00:00|    2600| 740000|       house| 

In [74]:
# Средняя стомость 10 проданных домов до текущего в том же районе (в pyspark)
w = Window().partitionBy('postcode')\
            .orderBy('datesold')\
            .rowsBetween(Window.currentRow - 9, Window.currentRow)

df.withColumn('avg_10_prev', F.mean('price').over(w)).show(15)

+-------------------+--------+-------+------------+--------+-----------------+
|           datesold|postcode|  price|propertyType|bedrooms|      avg_10_prev|
+-------------------+--------+-------+------------+--------+-----------------+
|2007-07-08 00:00:00|    2600| 327000|       house|       1|         327000.0|
|2007-08-16 00:00:00|    2600| 790000|       house|       4|         558500.0|
|2007-12-05 00:00:00|    2600| 825000|       house|       3|647333.3333333334|
|2008-01-21 00:00:00|    2600| 315000|        unit|       1|         564250.0|
|2008-04-24 00:00:00|    2600| 292500|       house|       1|         509900.0|
|2008-05-30 00:00:00|    2600| 329000|        unit|       2|         479750.0|
|2008-06-19 00:00:00|    2600| 765000|       house|       5|         520500.0|
|2008-07-29 00:00:00|    2600| 927000|       house|       4|         571312.5|
|2008-09-02 00:00:00|    2600|1380000|       house|       5|661166.6666666666|
|2008-09-08 00:00:00|    2600| 740000|       house| 

In [77]:
# Средняя стомость 10 проданных домов после текущего в том же районе (в SQL)
df.registerTempTable('df_task_1_2')

spark.sql('''
select *, 
AVG(price) OVER(PARTITION BY postcode ORDER BY datesold ROWS BETWEEN CURRENT ROW AND 9 FOLLOWING) AS avg_10_next
FROM df_task_1_2
''').show(15)

/usr/local/lib/python3.7/dist-packages/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


+-------------------+--------+-------+------------+--------+-----------+
|           datesold|postcode|  price|propertyType|bedrooms|avg_10_next|
+-------------------+--------+-------+------------+--------+-----------+
|2007-07-08 00:00:00|    2600| 327000|       house|       1|   669050.0|
|2007-08-16 00:00:00|    2600| 790000|       house|       4|   708350.0|
|2007-12-05 00:00:00|    2600| 825000|       house|       3|   698350.0|
|2008-01-21 00:00:00|    2600| 315000|        unit|       1|   679350.0|
|2008-04-24 00:00:00|    2600| 292500|       house|       1|   742850.0|
|2008-05-30 00:00:00|    2600| 329000|        unit|       2|   786600.0|
|2008-06-19 00:00:00|    2600| 765000|       house|       5|   839200.0|
|2008-07-29 00:00:00|    2600| 927000|       house|       4|   868450.0|
|2008-09-02 00:00:00|    2600|1380000|       house|       5|   805750.0|
|2008-09-08 00:00:00|    2600| 740000|       house|       3|   715250.0|
|2008-09-17 00:00:00|    2600| 720000|       house|

In [86]:
# Средняя стомость 10 проданных домов после текущего в том же районе (в pyspark)

w = Window().partitionBy('postcode')\
            .orderBy('datesold')\
            .rowsBetween(Window.currentRow, Window.currentRow + 9)

df.withColumn('avg_10_next', F.mean('price').over(w)).show(15)

+-------------------+--------+-------+------------+--------+-----------+
|           datesold|postcode|  price|propertyType|bedrooms|avg_10_next|
+-------------------+--------+-------+------------+--------+-----------+
|2007-07-08 00:00:00|    2600| 327000|       house|       1|   669050.0|
|2007-08-16 00:00:00|    2600| 790000|       house|       4|   708350.0|
|2007-12-05 00:00:00|    2600| 825000|       house|       3|   698350.0|
|2008-01-21 00:00:00|    2600| 315000|        unit|       1|   679350.0|
|2008-04-24 00:00:00|    2600| 292500|       house|       1|   742850.0|
|2008-05-30 00:00:00|    2600| 329000|        unit|       2|   786600.0|
|2008-06-19 00:00:00|    2600| 765000|       house|       5|   839200.0|
|2008-07-29 00:00:00|    2600| 927000|       house|       4|   868450.0|
|2008-09-02 00:00:00|    2600|1380000|       house|       5|   805750.0|
|2008-09-08 00:00:00|    2600| 740000|       house|       3|   715250.0|
|2008-09-17 00:00:00|    2600| 720000|       house|

In [85]:
# Стоимость последнего проданного дома до текущего (в SQL)
df.registerTempTable('df_task_1_3')

spark.sql('''
select *, 
LAG(price, 1, 0) OVER(ORDER BY datesold) AS prev_price
FROM df_task_1_3
''').show(15)

/usr/local/lib/python3.7/dist-packages/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


+-------------------+--------+-------+------------+--------+----------+
|           datesold|postcode|  price|propertyType|bedrooms|prev_price|
+-------------------+--------+-------+------------+--------+----------+
|2007-02-07 00:00:00|    2607| 525000|       house|       4|         0|
|2007-02-27 00:00:00|    2906| 290000|       house|       3|    525000|
|2007-03-07 00:00:00|    2905| 328000|       house|       3|    290000|
|2007-03-09 00:00:00|    2905| 380000|       house|       4|    328000|
|2007-03-21 00:00:00|    2906| 310000|       house|       3|    380000|
|2007-04-04 00:00:00|    2905| 465000|       house|       4|    310000|
|2007-04-24 00:00:00|    2607| 399000|       house|       3|    465000|
|2007-04-30 00:00:00|    2606|1530000|       house|       4|    399000|
|2007-05-24 00:00:00|    2902| 359000|       house|       3|   1530000|
|2007-05-25 00:00:00|    2906| 320000|       house|       3|    359000|
|2007-06-26 00:00:00|    2902| 385000|       house|       3|    

In [84]:
# Стоимость последнего проданного дома до текущего (в pyspark)

w = Window().orderBy('datesold')
df.withColumn('prev_price', F.lag('price', 1, 0).over(w)).show(15)

+-------------------+--------+-------+------------+--------+----------+
|           datesold|postcode|  price|propertyType|bedrooms|prev_price|
+-------------------+--------+-------+------------+--------+----------+
|2007-02-07 00:00:00|    2607| 525000|       house|       4|         0|
|2007-02-27 00:00:00|    2906| 290000|       house|       3|    525000|
|2007-03-07 00:00:00|    2905| 328000|       house|       3|    290000|
|2007-03-09 00:00:00|    2905| 380000|       house|       4|    328000|
|2007-03-21 00:00:00|    2906| 310000|       house|       3|    380000|
|2007-04-04 00:00:00|    2905| 465000|       house|       4|    310000|
|2007-04-24 00:00:00|    2607| 399000|       house|       3|    465000|
|2007-04-30 00:00:00|    2606|1530000|       house|       4|    399000|
|2007-05-24 00:00:00|    2902| 359000|       house|       3|   1530000|
|2007-05-25 00:00:00|    2906| 320000|       house|       3|    359000|
|2007-06-26 00:00:00|    2902| 385000|       house|       3|    

## Задание 2
В итоге у вас таблица с колонками (или нечто похожее):
*   price
*   Среднегодовая цена
*  Средняя стомость 10 проданных домов до текущего в том же районе (4digit postcode) (1 балл)
*  Средняя стомость 10 проданных домов после текущего в том же районе (4digit postcode) (1 балл)
*  Стоимость последнего проданного дома до текущего ((1 балл)
*  и др.

Посчитайте кол-во уникальных значений в каждой строчке (unique(row)) (ипользуйте udf). Попробуйте сделать то же самое используя pandas udf.

In [170]:
w_y = Window().orderBy('year')

w_prev = Window().partitionBy('postcode')\
            .orderBy('datesold')\
            .rowsBetween(Window.currentRow - 9, Window.currentRow)

w_next = Window().partitionBy('postcode')\
            .orderBy('datesold')\
            .rowsBetween(Window.currentRow, Window.currentRow + 9)

w = Window().orderBy('datesold')

df = df.withColumn('year', F.split("datesold", "-").getItem(0))\
       .withColumn('mean', F.mean('price').over(w_y))\
       .withColumn('avg_10_prev', F.mean('price').over(w_prev))\
       .withColumn('avg_10_next', F.mean('price').over(w_next))\
       .withColumn('prev_price', F.lag('price', 1, 0).over(w))

df.show(15)

+-------------------+--------+-------+------------+--------+----+-----------------+-----------------+-----------+----------+
|           datesold|postcode|  price|propertyType|bedrooms|year|             mean|      avg_10_prev|avg_10_next|prev_price|
+-------------------+--------+-------+------------+--------+----+-----------------+-----------------+-----------+----------+
|2007-07-08 00:00:00|    2600| 327000|       house|       1|2007|522377.2108843537|         327000.0|   669050.0|    900000|
|2007-08-16 00:00:00|    2600| 790000|       house|       4|2007|522377.2108843537|         558500.0|   708350.0|    625000|
|2007-12-05 00:00:00|    2600| 825000|       house|       3|2007|522377.2108843537|647333.3333333334|   698350.0|    620000|
|2008-01-21 00:00:00|    2600| 315000|        unit|       1|2008| 499156.106870229|         564250.0|   679350.0|    580000|
|2008-04-24 00:00:00|    2600| 292500|       house|       1|2008| 499156.106870229|         509900.0|   742850.0|    445000|


In [143]:
l = lambda row: '**'.join(row) # объединим все ячейки
l_udf = F.udf(l, returnType='string')

df.select(l_udf(F.array_distinct(F.array('*')))).count() # посчитаем кол-во уникальных

29580

In [149]:
l = lambda row: '**'.join(row) # объединим все ячейки
l_udf = F.pandas_udf(l, returnType='string')

df.select(l_udf(F.array_distinct(F.array('*')))).count() # посчитаем кол-во уникальных

29580

# Задание 3
SQL like case when или if elif else

Создайте колонку, в которой в которой будет отображаться "+", "-" или "=", если "Средняя стомость 10 проданных домов до текущего в том же районе" больше, меньше или равно "Средняя стомость 10 проданных домов после текущего в том же районе (4digit postcode)", соотвественно.

Если одно из полей Null, запишите в эту колонку "Нет данных"

In [175]:
df.withColumn('compare', F.when(F.col('avg_10_prev') < F.col('avg_10_next'), F.lit('+'))\
                          .when(F.col('avg_10_prev') > F.col('avg_10_next'), F.lit('-'))\
                          .otherwise('='))\
             .show(15)

+-------------------+--------+-------+------------+--------+----+-----------------+-----------------+-----------+----------+-------+
|           datesold|postcode|  price|propertyType|bedrooms|year|             mean|      avg_10_prev|avg_10_next|prev_price|compare|
+-------------------+--------+-------+------------+--------+----+-----------------+-----------------+-----------+----------+-------+
|2007-07-08 00:00:00|    2600| 327000|       house|       1|2007|522377.2108843537|         327000.0|   669050.0|    900000|      +|
|2007-08-16 00:00:00|    2600| 790000|       house|       4|2007|522377.2108843537|         558500.0|   708350.0|    625000|      +|
|2007-12-05 00:00:00|    2600| 825000|       house|       3|2007|522377.2108843537|647333.3333333334|   698350.0|    620000|      +|
|2008-01-21 00:00:00|    2600| 315000|        unit|       1|2008| 499156.106870229|         564250.0|   679350.0|    580000|      +|
|2008-04-24 00:00:00|    2600| 292500|       house|       1|2008| 499

# Задание 4

In [176]:
# Создаём датасет для примеров
dataset_1 = [
  {
    'key_1' : 'abc',
    'value_1' : 10,
    'value_2' : 20
  },
  {
    'key_1' : 'def',
    'value_1' : 100,
    'value_2' : 300
  }
]

dataset_2 = [
  {
    'key_2' : 'abc',
    'value_1' : 5.5,
    'value_2' : 2.2
  },
  {
    'key_2' : 'xyz',
    'value_1' : 10.1,
    'value_2' : 13.5
  }
]

df1 = spark.createDataFrame(dataset_1)
print('df1')
df1.show()

df2 = spark.createDataFrame(dataset_2)
print('df2')
df2.show()

df1
+-----+-------+-------+
|key_1|value_1|value_2|
+-----+-------+-------+
|  abc|     10|     20|
|  def|    100|    300|
+-----+-------+-------+

df2
+-----+-------+-------+
|key_2|value_1|value_2|
+-----+-------+-------+
|  abc|    5.5|    2.2|
|  xyz|   10.1|   13.5|
+-----+-------+-------+



## Задание 4.1

In [ ]:
# Создайте джойн, чтобы получить следующую таблицу
# +---+-------+-------+
# |key|value_1|value_2|
# +---+-------+-------+
# |abc|     10|     20|
# +---+-------+-------+

In [182]:
df1.join(df2, on=df1.key_1 == df2.key_2, how='leftsemi').show()

+-----+-------+-------+
|key_1|value_1|value_2|
+-----+-------+-------+
|  abc|     10|     20|
+-----+-------+-------+



## Задание 4.2

In [ ]:
# Создайте джойн, чтобы получить следующую таблицу
# +---+-------+-------+
# |key|value_1|value_2|
# +---+-------+-------+
# |def|    100|    300|
# +---+-------+-------+

In [183]:
df1.join(df2, on=df1.key_1 == df2.key_2, how='leftanti').show()

+-----+-------+-------+
|key_1|value_1|value_2|
+-----+-------+-------+
|  def|    100|    300|
+-----+-------+-------+



## Задание 4.3

In [ ]:
# Создайте Inner джойн с условиями ---hidden---, для df1 и df2, соответсвенно
# В  итоге получится таблица
# +---+-------+-------+---+-------+-------+
# |key|value_1|value_2|key|value_1|value_2|
# +---+-------+-------+---+-------+-------+
# |abc|     10|     20|abc|    5.5|    2.2|
# |abc|     10|     20|xyz|   10.1|   13.5|
# |def|    100|    300|abc|    5.5|    2.2|
# |def|    100|    300|xyz|   10.1|   13.5|
# +---+-------+-------+---+-------+-------+

In [184]:
df1.join(df2, on=df1.value_2 > df2.value_2, how='inner').show()

+-----+-------+-------+-----+-------+-------+
|key_1|value_1|value_2|key_2|value_1|value_2|
+-----+-------+-------+-----+-------+-------+
|  abc|     10|     20|  abc|    5.5|    2.2|
|  abc|     10|     20|  xyz|   10.1|   13.5|
|  def|    100|    300|  abc|    5.5|    2.2|
|  def|    100|    300|  xyz|   10.1|   13.5|
+-----+-------+-------+-----+-------+-------+

